# Test Microsoft Presidio on all data for binary classification

In [1]:
import pandas as pd
import ast
from presidio_analyzer import AnalyzerEngine
import json
from presidio_structured import PandasAnalysisBuilder

# Dataset containing mimesis, faker and dessi with unique values for specific classes

In [2]:
all_data_test = pd.read_csv("../../datasets/dessi-mf/dessi-mf/test.csv")
all_data_labels = pd.read_csv("../../datasets/dessi-mf/dessi-mf/test_labels_multi.csv").T
all_data_test.head()

,f4WINK6i,credit_card_number_fr_80,company_en_266,credit_card_provider_fr_FR_308,MAidccS9b5Stvo,eO5TDsh,swift_mixed_110,pyfloat_de_DE_116,Xp0yUeIVQAP3wHll,OeSIJzFoOtDJ,...,geo_lon.14,user_agent_mixed_20,3iwJq,yrCXaX,email_mixed_186,blood_type_fr_208,email_en_50,z6r0fbtbI3aJtXFu5ON,hwdskaedozyz,version_en_22
0,9150,4565 7937 2995 3231,Wu LLC,VISA 16 digit,Master,Anarchism,BCMNDE0B,4.276553e+07,38768386629875,marcelfernandes@example.com,...,39.3691433; -16.2714811,Opera/9.63 (X11; FreeBSD 7.1-RELEASE i386; U; ...,1280x768,2813943257367,cingular2059@yahoo.com,A−,andersonnicole@example.net,Herr,23 Oct 79,23.65.20
1,2566,3772 946671 16846,Meadows and Sons,JCB 16 digit,Bachelor,Anarchism,LPNMFRJNT22,-7.880751e+01,4331819986949,penelopevaillant@example.org,...,28.6479388 -74.6576142,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,1400x1050,2905471822169,print1989@example.org,O−,joneskathleen@example.org,Frau,1992-08-12,50.97.23
2,3909,4532 0710 1134 6723,"Harrell, White and Bowers",Mastercard,Master,Anarchism,ZZPBGBUW,7.571927e+03,30322452662309,augustinmasson@example.com,...,"28.4832921, -74.6063272",Mozilla/5.0 (Windows NT 6.3; Win64; x64) Apple...,1600x1200,4848935006492,seek2100@yahoo.com,O−,branchkayla@example.net,Herr,2009-03-24,18.70.75
3,4574,3469 556889 86403,Hernandez Inc,VISA 16 digit,PhD,Socialist,VSQBFR61,8.427374e+12,30553960738559,chantal65@example.net,...,"39.4176592, -17.7753623",Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,1152x768,6917967769311,girl1976@yandex.com,B+,tiffanyharris@example.com,Dr. rer. nat.,15 Sep 10,8.85.54
4,8729,3420 971634 58790,Wagner Ltd,Diners Club / Carte Blanche,Bachelor,Anarchism,WWHXDEKU,-2.256568e+10,4110303952895157716,mpascal@example.net,...,"28.9629615, -74.7948488",Mozilla/5.0 (Linux; Android 5.0.1; SM-N910V Bu...,1280x768,988109762602,victim2046@duck.com,O−,dominique34@example.com,Herr,Jan 1 12,20.28.34


In [3]:
#only execute first time
results_rowwise1 = all_data_labels.copy()
results_rowwise1.to_csv("binary_results/dessi-mf/results_rowwise1.csv", index=False)

Method 1: 
- Input: The values are concatenated with the column names like a dictionary, the delimiter is a ,  
Example: "0: Email, 1: Phone_number, 2: Email, ...
- Output: Based on the position in the string where personal data is detected the detected classes are matched to the columns

In [4]:
def convert_to_text_rowwise1(df, row):
    s_array = []
    string_position_array = []
    s = ""
    string_position = []
    start_index = 0
    end_index = -1
    for e, c in enumerate(df.columns):
        add_string = str(c) + ": " + str(df.loc[row,c]) + ", "
        s = s + add_string
        start_index = end_index + 1 
        end_index = start_index + len(add_string) - 1
        string_position.append((start_index, end_index))
        if (e+1) % 100 == 0:
            s_array.append(s)
            string_position_array.append(string_position)
            s = ""
            string_position = []
    if string_position != []:
        s_array.append(s)
        string_position_array.append(string_position) 
    return s_array, string_position_array

In [5]:
def apply_analyzer(a, file_path, convert_method, data):                  #a is the row where the analysis starts due to several kernel crashes    
    results = pd.read_csv(file_path)
    analyzer = AnalyzerEngine(supported_languages=["en", "hr"])               # other nlp_engine possible, support english and croatian
    for i in range(a,100):
        print(i)
        position_index = i*2+1
        entities_index = i*2+2
        data_string, string_position = convert_method(data, i)
        results.loc[position_index] = [item for sublist in string_position for item in sublist]
        empty_row = [[] for _ in range(results.shape[1])]
        results.loc[entities_index] = empty_row
        last_stringposition = 0
        for e, substring in enumerate(data_string):
            results_analyzer = analyzer.analyze(text=substring, language="en")                #multiple languages could be a problem, for analyze is only one language possible
            column = 0
            results_analyzer.sort(key=lambda x: x.start)
            for result in results_analyzer:
                result_tuple = (result.start + last_stringposition, result.end + last_stringposition- 1)               # result.end gives the next character after the entity
                string_tuple = results.iloc[position_index, column]
                while result_tuple[0] > string_tuple[1]:                    #case if there is no more entity in the column
                    column += 1
                    string_tuple = results.iloc[position_index, column]
                if  string_tuple[0] <= result_tuple[0] and result_tuple[1] <= string_tuple[1]:      #case if entity is one column
                    results.iloc[entities_index,column].append(str(result.entity_type) + "_" + str(result.score)) 
                else:
                    counter = 0 
                    while True:
                        results.iloc[entities_index,column+counter].append("shared " + str(result.entity_type) + "_" + str(result.score))             # case if entity is in multiple columns
                        counter += 1
                        if column+counter > results.shape[1]:
                            break
                        string_tuple = results.iloc[position_index, column+counter]
                        if result_tuple[1] < string_tuple[0]:
                            break
            last_stringposition = string_position[e][-1][1] + 1
        results.to_csv(file_path, index=False)

In [ ]:
apply_analyzer(0, "binary_results/dessi-mf/results_rowwise1.csv", convert_to_text_rowwise1, all_data_test)

In all results dataframes the first row contains the groundtruth label, the other rows contain the string position of the corresponding cell value and underneath the detected personal classed for this cell value

Method 2: Only use the cell values without the column names
- Input: Only use the cell values without the column names, the delimiter is a ,  
Example: "Email, Phone_number, Email, ...
- Output: Based on the position in the string where personal data is detected the detected classes are matched to the columns

In [7]:
#only execute first time
results_rowwise2 = all_data_labels.copy()
results_rowwise2.to_csv("binary_results/dessi-mf/results_rowwise2.csv", index=False)

In [8]:
def convert_to_text_rowwise2(df, row):
    s_array = []
    string_position_array = []
    s = ""
    string_position = []
    start_index = 0
    end_index = -1
    for e, c in enumerate(df.columns):
        add_string = str(df.loc[row,c]) + ", "
        s = s + add_string
        start_index = end_index + 1 
        end_index = start_index + len(add_string) - 1
        string_position.append((start_index, end_index))
        if (e+1) % 100 == 0:
            s_array.append(s)
            string_position_array.append(string_position)
            s = ""
            string_position = []
    if string_position != []:
        s_array.append(s)
        string_position_array.append(string_position) 
    return s_array, string_position_array

In [9]:
apply_analyzer(0, "binary_results/dessi-mf/results_rowwise2.csv", convert_to_text_rowwise2, all_data_test)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


Method3: 
- Input: Columnwise approach, pass the column name and all cell values to the Analyzer, the delimiters are : and ,  
Example: "0: Email, Email, Email, Email, ..."
- Output: All detected classes can be matched to the analyzed column, string positions are not important but are saved anyways for the cell values

In [26]:
# only execute first time
results_columnwise = all_data_labels.copy()
df_add = pd.DataFrame([[[] for _ in range(all_data_labels.shape[1])] for _ in range(200)])
df_add.columns = results_columnwise.columns
results_columnwise = pd.concat([results_columnwise, df_add]).reset_index(drop=True)
results_columnwise.to_csv("./binary_results/dessi-mf/results_columnwise.csv", index=False)

In [27]:
def convert_to_text_columnwise(df, column):
    s = str(column) + ": "
    string_position = pd.Series(dtype="object")
    start_index = 0
    end_index = len(s) - 1
    for r in range(100):
        add_string = str(df.loc[r,column]) + ", "
        s = s + add_string
        start_index = end_index + 1 
        end_index = start_index + len(add_string) - 1
        string_position.loc[r] = (start_index, end_index)
    return s, string_position

In [28]:
def apply_analyzer_columnwise(a, file_path, dessi):                  #a is the row where the analysis starts due to several kernel crashes    
    results = pd.read_csv(file_path)
    analyzer = AnalyzerEngine(supported_languages=["en", "hr"])               # other nlp_engine possible, support english and croatian
    for i in range(a,results.shape[1]):
        print(i)
        column = dessi.columns[i]
        dessi_string, string_position = convert_to_text_columnwise(dessi, column)
        for j in range(1,201,2):
            results.iloc[j, i] = string_position[(j-1)/2]
        results_analyzer = analyzer.analyze(text=dessi_string, language="en")                #multiple languages could be a problem, for analyze is only one language possible

        results_analyzer.sort(key=lambda x: x.start)
        row = 0
        for result in results_analyzer:
            result_tuple = (result.start, result.end - 1)               # result.end gives the next character after the entity
            string_tuple = results.iloc[row*2+1, i]
            while result_tuple[0] > string_tuple[1]:                    #case if there is no more entity in the column
                row += 1
                string_tuple = results.iloc[row*2+1, i]
            if  string_tuple[0] <= result_tuple[0] and result_tuple[1] <= string_tuple[1]:      #case if entity is one column
                results.iloc[row*2+2, i] = ast.literal_eval(str(results.iloc[row*2+2, i])) + [str(result.entity_type) + "_" + str(result.score)]
            else:
                counter = 0 
                while True:
                    results.iloc[(row+counter)*2+2, i] = ast.literal_eval(str(results.iloc[(row+counter)*2+2, i])) + ["shared " + str(result.entity_type) + "_" + str(result.score)]             # case if entity is in multiple columns
                    counter += 1
                    if row+counter > 99:
                            break
                    string_tuple = results.iloc[(row+counter)*2+1, i]
                    if result_tuple[1] < string_tuple[0]:
                        break
        if i % 100 == 0:
            results.to_csv(file_path, index=False)
    results.to_csv(file_path, index=False)
            

In [ ]:
apply_analyzer_columnwise(0, "./binary_results/dessi-mf/results_columnwise.csv", all_data_test)

# Presidio structured

In [14]:
def apply_presidio_structured(data, path, selection_strategy="most_common", a=0):
    if a != 0:
        with open(path, "rb") as file:
            tabular_analysis = json.load(file)
    else:
        tabular_analysis = dict()
    for i in range(a, data.shape[1], 100):
        print(i)
        tabular_analysis_partly = PandasAnalysisBuilder().generate_analysis(data.iloc[:, i:i+100], selection_strategy=selection_strategy)
        tabular_analysis = tabular_analysis | tabular_analysis_partly.entity_mapping
        with open(path, "w") as file:
            json.dump(tabular_analysis, file)

In [15]:
apply_presidio_structured(all_data_test, 'binary_results/dessi-mf/most_common.json')

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700


In [16]:
apply_presidio_structured(all_data_test, 'binary_results/dessi-mf/highest_confidence.json', selection_strategy="highest_confidence")

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
